In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.dtypes

,0
isbn,object
title,object
author,object


In [6]:
user_ratings_count = df_ratings.groupby('user').size()
book_ratings_count = df_ratings.groupby('isbn').size()


In [7]:
df_ratings = df_ratings[df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(book_ratings_count[book_ratings_count >= 100].index)]


In [25]:
merged_df = pd.merge(right=df_ratings, left=df_books, on="isbn")
merged_df = merged_df.drop_duplicates(subset=["title", "user"])
df_pivot = merged_df.pivot(index="title", columns="user", values="rating").fillna(0)
user_item_matrix = csr_matrix(df_pivot.values)

In [56]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [57]:
def get_recommends(book=""):
    # Find the index of the book in the dataframe
    book_idx = df_pivot.values[(list(df_pivot.index.values)).index(book), :]
    # print(book_idx)
    # Find the nearest neighbors (5 most similar books)
    distances, indices = model.kneighbors(np.reshape(book_idx, [1, -1]), n_neighbors=6)  # 6 because it includes the book itself

    # Prepare the list of recommended books with their distances
    recommended_books = []
    for i in range(1, len(indices.flatten())):  # start from 1 to exclude the book itself
        similar_book_title =  list(df_pivot.index.values)[indices.flatten()[i]]
        recommended_books.append([similar_book_title, distances.flatten()[i]])
    # Sort the recommended books by distance in decreasing order
    recommended_books.sort(key=lambda x: x[1], reverse=True)


    return [book, recommended_books]


In [58]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
